In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from sklearn.feature_extraction.text import CountVectorizer

In [2]:
import numpy as np

In [3]:
driver = webdriver.Chrome('your chrome driver adress')
driver.maximize_window()

/var/folders/3m/s0ysqyzs3fb_mcgvm4wjzt3w0000gn/T/ipykernel_62796/1366764065.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/jonghyunkim/repo/Tristan/chromedriver')


In [4]:
url = 'https://sdgs.un.org/partnerships/browse'
driver.get(url)

In [5]:
def scroll_down_and_click(driver, element):
    while True:
        try:
            element.click()
            break
        except:
            driver.find_element(By.CSS_SELECTOR, 'body').send_keys(Keys.ARROW_DOWN)

In [6]:
case_names = []
descrip_tab = []
sdgs_tab = []
deliver_tab = []
resource_mobilized_tab = []
progress_tab = []

right_hand_info = []

k = 1
numbers = 1

errors = []

while True:
    cases_in_page = driver.find_element(By.CLASS_NAME, 'view-content').find_elements(By.CLASS_NAME, 'views-row')

    for case in cases_in_page:
        # 가끔 클릭했는데 페이지가 안넘어가는 애들이 있어서 얘네는 따로 저장해주는 걸로.
        try:
            case2 = case.find_elements(By.CLASS_NAME, 'views-field')[0]
            scroll_down_and_click(driver, case2)
            time.sleep(5)
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            # 케이스의 이름 - identifier
            case_name = soup.find(class_ = 'separator-bottom mt-5').string.strip()
        except:
            errors.append((numbers, case2.text))
            numbers += 1
            continue
        case_names.append(case_name)



        #Description Tab
        description_tab = soup.find_all(class_ = 'details-wrapper')[0]
        ## Intro 랑 Description만 필요
        contents_names = description_tab.find_all(class_ = 'field__label')
        contents_des = description_tab.find_all(class_ = 'field__item')
        descripts = []

        i = 0
        while i < len(contents_names):
            if contents_names[i].text.strip() in ['Intro', 'Description']:
                descripts.append((contents_names[i].text.strip(), contents_des[i].text.strip()))
            
            i += 1

        descrip_tab.append(descripts)

        #SDGS & Targets Tab
        sdgs_targets_tab = soup.find_all(class_ = 'details-wrapper')[1]

        GOALS = [sdgs_targets_tab.find_all(class_ = 'taxonomy-term container vocabulary-goals')[n].find(class_ = 'goal-title').text.strip() for n in range(len(sdgs_targets_tab.find_all(class_ = 'taxonomy-term container vocabulary-goals')))]
        sdgs_tab.append(GOALS)



        # Deliverables&Timeline tab
        deliverable_tab = soup.find_all(class_ = 'details-wrapper')[3]
        
        
        time_line = deliverable_tab.find_all(class_ = 'datetime')
        time_description = deliverable_tab.find_all(class_ = 'clearfix text-formatted field field--name-field-description-of-deliverable field--type-text-long field--label-hidden field__item')

        i = 0
        delivers = []

        while i < len(time_line):
            delivers.append((time_line[i].text.strip(), time_description[i].text.strip()))
            i += 1
        deliver_tab.append(delivers)



        # Resources Mobilized
        resources_tab = soup.find_all(class_ = 'details-wrapper')[4]
        the_resource = resources_tab.find_all(class_ = 'field field--name-field-type field--type-list-string field--label-hidden field__item')
        
        resource_description = resources_tab.find_all(class_ = 'field field--name-field-details field--type-string field--label-hidden field__item')
        
        i = 0
        resources = []
        while i < len(the_resource):
            resources.append((the_resource[i].text.strip(), resource_description[i].text.strip() if len(resource_description) != 0 else None))
            i += 1
        
        resource_mobilized_tab.append(resources)


        #Progress Reports
        progress_report_tqb = soup.find_all(class_ = 'details-wrapper')[5]
        progress_titles = progress_report_tqb.find_all(class_ = 'views-field views-field-title')
        progress_status = progress_report_tqb.find_all(class_ = 'views-field views-field-field-progress-status')
        progress_filing = progress_report_tqb.find_all(class_ = 'views-field views-field-created')

        i = 1
        progress_info = []
        while i < len(progress_titles):
            progress_info.append((progress_titles[i].text.strip(), progress_status[i].text.strip(), progress_filing[i].text.strip()))
            i += 1

        progress_tab.append(progress_info)


        
        ## Right_hand Side
        right_side = soup.find(class_ = 'partnerships-basic-information-block container mt-5')
        containers = right_side.find_all(class_ = 'views-element-container')
        rights = []

        for x in containers:
            temp = []
            if x.text.strip().split('\n')[0] in ['Region', 'Geographical coverage', 'Countries']:
                pure = [k for k in x.text.strip().split('\n') if (len(k) > 0) & (k != ' ')]
                temp.append((pure[0], pure[1:]))
            if len(temp) != 0:
                rights.append(temp)

        if len(rights) != 0:
            right_hand_info.append(rights)
        else:
            right_hand_info.append(np.nan)

        driver.back()
        time.sleep(3)
        # l += 1
        numbers += 1
    next_page_path = [x for x in driver.find_elements(By.CLASS_NAME, 'page-item') if x.text.strip() == str(k+1)][0]
    if k == 448:
        break
    scroll_down_and_click(driver, next_page_path)
    # driver.find_elements(By.CLASS_NAME, 'page-item')[k].click()
    time.sleep(5)
    k += 1
    # 중간에 끊는애
    # if numbers >= 100:
    #     break
 
    

In [8]:
errors

[(9, 'AQUALEX'),
 (38, 'Maison IAGU'),
 (39, 'Raw Revel'),
 (42, 'Fiume'),
 (44, 'Zirkulu'),
 (48, 'IMPACT NYFW'),
 (63, 'Saheli Women'),
 (77, 'Cellsense'),
 (79, 'CSFDG+'),
 (80, 'WASTEWEAR'),
 (84, 'Dunia Bora'),
 (86, 'Bliss Lau'),
 (98, 'MM Milano'),
 (99, 'UZURI K&Y'),
 (100, 'Materia CS'),
 (104, 'North Sails'),
 (107, 'Sellery'),
 (108, 'Earth Partner')]

In [39]:
error_nums = []
for x,y in errors:
    error_nums.append(x)
error_nums

[9, 38, 39, 42, 44, 48, 63, 77, 79, 80, 84, 86, 98, 99, 100, 104, 107, 108]

In [9]:
print(len(case_names))
print(len(descrip_tab))
print(len(sdgs_tab))
print(len(deliver_tab))
print(len(resource_mobilized_tab))
print(len(progress_tab))

print(len(right_hand_info))

90
90
90
90
90
90
90


In [10]:
deliver_tab[0]

[('31 December, 2024',
  'Run one complete program in one of the partner universities.'),
 ('01 December, 2024',
  'Enable 3000 students to do a school-to-work transition and enter labour market'),
 ('20 January, 2024',
  'Establish at least one partnership to create new educational content, in particular on Circular Economy and Social Entrepreneurship'),
 ('01 February, 2021',
  'Involve 60 students in connecting with local businesses and analyzing their challenges'),
 ('01 August, 2020',
  'Involve 18 students and 5 business into the pilot of the Lockdown Economy Challenge'),
 ('01 November, 2021',
  'Test the series of mini classes with 30 volunteers involved in Lockdown Economy'),
 ('01 September, 2021',
  'Record a series of 10 mini classes educating graduates about the job search process')]

In [18]:
right_hand_info

[[[('Region', ['Global'])],
  [('Geographical coverage', ['Multiple locations around the world'])],
  [('Countries',
    ['Albania',
     'Colombia',
     'Italy',
     'Mexico',
     'Netherlands',
     'Romania',
     'Spain',
     'Venezuela, Bolivarian Republic of'])]],
 [[('Region', ['West Asia'])],
  [('Geographical coverage', ['Shirvan, Iran'])],
  [('Countries', ['Climate Emergency Institute'])]],
 [[('Region', ['West Asia'])],
  [('Geographical coverage', ['Shirvan, Iran'])],
  [('Countries', ['Climate Emergency Institute'])]],
 [[('Region',
    ['AfricaEuropeAsia and PacificLatin America and the Caribbean'])],
  [('Geographical coverage', ['Madrid, Spain'])],
  [('Countries',
    ['Chile', 'Colombia', 'Ethiopia', 'Ghana', 'Spain', 'Thailand'])]],
 [[('Geographical coverage', ['Stockholm, Sweden'])],
  [('Countries', ['\t\t\t\t\t\t\tN/A'])]],
 [[('Region', ['Global'])],
  [('Geographical coverage', ['Global'])],
  [('Countries', ['China', 'Ghana', 'Kenya', 'Sri Lanka', 'Zambia

In [59]:
series_storage = []

i = 0
j = 0
k = 0
while i < 108:

    if (i + 1) in error_nums:
        case = errors[k][1]
        description_intro = 'error'
        description_des = 'error'
        sdgs = 'error'
        delivery_date = 'error'
        delivery_des = 'error'
        resource_name = 'error'
        resource_des = 'error'
        progress_t = 'error'
        progress_s = 'error'
        progress_f = 'error'
        region = 'error'
        geo_cover = 'error'
        countries = 'error'
        k += 1
    else:
        case = case_names[j]
    
        if len(descrip_tab[j]) == 0:
            description_intro = np.nan
            description_des = np.nan
        elif len(descrip_tab[j]) == 1:
            if descrip_tab[j][0][0] == 'Intro':
                description_intro = descrip_tab[j][0][1]
                description_des = np.nan
            elif descrip_tab[j][0][0] == 'Description':
                description_intro = np.nan
                description_des = descrip_tab[j][0][1]
        else:
            if descrip_tab[j][0][0] == 'Intro':
                description_intro = descrip_tab[j][0][1]
                description_des = descrip_tab[j][1][1]
            elif descrip_tab[j][0][0] == 'Description':
                description_intro = descrip_tab[j][1][1]
                description_des = descrip_tab[j][0][1]

        sdgs = sdgs_tab[j]

        delivery_date = []
        delivery_des = []

        for x,y in deliver_tab[j]:
            delivery_date.append(x)
            delivery_des.append(y)

        resource_name = []
        resource_des = []

        for x,y in resource_mobilized_tab[j]:
            resource_name.append(x)
            resource_des.append(y)

        progress_t = []
        progress_s = []
        progress_f = []
        
        for x,y,z in progress_tab[j]:
            progress_t.append(x)
            progress_s.append(y)
            progress_f.append(z)
        

        right_dict = {}

        for element in right_hand_info[j]:
            target = element[0]
            right_dict[target[0]] = target[1]
        
        for word in ['Region', 'Geographical coverage', 'Countries']:
            if word not in right_dict.keys():
                right_dict[word] = np.NaN
            else:
                pass

        region = right_dict['Region']
        geo_cover = right_dict['Geographical coverage']
        countries = right_dict['Countries']
        

        j += 1

        

        
    i += 1
                





    series = pd.Series({'Case_Name' : case, 'Description_Intro' : description_intro, 'Description_Description' : description_des,
               'SDGS&TARGETS' : sdgs, 'Delivery_Date' :  delivery_date, 'Delivery_Description' : delivery_des, 'Resource_Name': resource_name,
               'Resource_Description': resource_des, 'Progress_Title': progress_t, 'Progress_Status': progress_s, 'Progress_Filing': progress_f,
               'Region' : region, 'Geographical Coverage': geo_cover, 'Countries':countries})

    series_storage.append(series)


In [60]:
data = pd.DataFrame(series_storage)
data.head()

,Case_Name,Description_Intro,Description_Description,SDGS&TARGETS,Delivery_Date,Delivery_Description,Resource_Name,Resource_Description,Progress_Title,Progress_Status,Progress_Filing,Region,Geographical Coverage,Countries
0,AlterContacts Academy for Sustainable Development,The AlterContacts Academy for Sustainable Deve...,The Academy tackles three key societal issues:...,"[Goal 4, Goal 8, Goal 9, Goal 17, Goal 10, Goa...","[31 December, 2024, 01 December, 2024, 20 Janu...",[Run one complete program in one of the partne...,"[Staff / Technical expertise, In-kind contribu...",[Full time coordinators from Think Tank AlterC...,[AlterContacts Academy for Sustainable Develop...,[On track],"[20 March, 2024]",[Global],[Multiple locations around the world],"[Albania, Colombia, Italy, Mexico, Netherlands..."
1,Embroidering Impact: A Harmony of Style and So...,NaN,The primary objective of the DooziStyle initia...,"[Goal 1, Goal 4, Goal 5, Goal 8, Goal 10, Goal...","[01 January, 2025, 01 January, 2030, 01 Januar...","[Artisan Network Expansion Program:, Diversifi...","[Financing (in USD), Staff / Technical experti...","[About $200,000 through a combination of self-...",[],[],[],[West Asia],"[Shirvan, Iran]",[Climate Emergency Institute]
2,"Needle to Narrative, Empowering Rural Communit...",NaN,"Dooziba, founded by Fatemeh Abdi Ghoushbolagh ...","[Goal 1, Goal 4, Goal 5, Goal 8, Goal 10, Goal...","[01 January, 2025, 01 January, 2027, 01 Januar...",[Expanding Artisan Network: Dooziba aims to ex...,"[Financing (in USD), Staff / Technical experti...","[Amount: $200,000 Description: Secured financi...",[],[],[],[West Asia],"[Shirvan, Iran]",[Climate Emergency Institute]
3,The PET Lamp project,NaN,The PET Lamp project is a beautiful melding of...,"[Goal 1, Goal 5, Goal 8, Goal 10, Goal 12, Goa...","[03 December, 2024, 03 December, 2024]","[Portfolio/Dossier of PET Lamp, A portfolio wh...",[Staff / Technical expertise],[Experienced and talented teams worldwide based],[],[],[],[AfricaEuropeAsia and PacificLatin America and...,"[Madrid, Spain]","[Chile, Colombia, Ethiopia, Ghana, Spain, Thai..."
4,Material Exchange Ventures AB,NaN,Material Exchange works with the world's leadi...,"[Goal 9, Goal 5, Goal 17, Goal 12, Goal 13]","[01 January, 1970]",[Digital Sourcing Report],"[Staff / Technical expertise, In-kind contribu...",[The Chief Sustainability Officer drives the t...,[],[],[],NaN,"[Stockholm, Sweden]",[\t\t\t\t\t\t\tN/A]


In [61]:
len(data.columns)

14

In [62]:
len(data.iloc[1,8])

0

In [63]:
i = 0

while i < len(data):
    j = 0
    while j < len(data.columns):
        if (type(data.iloc[i,j]) == list):
            if len(data.iloc[i,j]) == 0:
                data.iloc[i,j] = np.NaN
        j += 1
    i += 1
data

,Case_Name,Description_Intro,Description_Description,SDGS&TARGETS,Delivery_Date,Delivery_Description,Resource_Name,Resource_Description,Progress_Title,Progress_Status,Progress_Filing,Region,Geographical Coverage,Countries
0,AlterContacts Academy for Sustainable Development,The AlterContacts Academy for Sustainable Deve...,The Academy tackles three key societal issues:...,"[Goal 4, Goal 8, Goal 9, Goal 17, Goal 10, Goa...","[31 December, 2024, 01 December, 2024, 20 Janu...",[Run one complete program in one of the partne...,"[Staff / Technical expertise, In-kind contribu...",[Full time coordinators from Think Tank AlterC...,[AlterContacts Academy for Sustainable Develop...,[On track],"[20 March, 2024]",[Global],[Multiple locations around the world],"[Albania, Colombia, Italy, Mexico, Netherlands..."
1,Embroidering Impact: A Harmony of Style and So...,NaN,The primary objective of the DooziStyle initia...,"[Goal 1, Goal 4, Goal 5, Goal 8, Goal 10, Goal...","[01 January, 2025, 01 January, 2030, 01 Januar...","[Artisan Network Expansion Program:, Diversifi...","[Financing (in USD), Staff / Technical experti...","[About $200,000 through a combination of self-...",NaN,NaN,NaN,[West Asia],"[Shirvan, Iran]",[Climate Emergency Institute]
2,"Needle to Narrative, Empowering Rural Communit...",NaN,"Dooziba, founded by Fatemeh Abdi Ghoushbolagh ...","[Goal 1, Goal 4, Goal 5, Goal 8, Goal 10, Goal...","[01 January, 2025, 01 January, 2027, 01 Januar...",[Expanding Artisan Network: Dooziba aims to ex...,"[Financing (in USD), Staff / Technical experti...","[Amount: $200,000 Description: Secured financi...",NaN,NaN,NaN,[West Asia],"[Shirvan, Iran]",[Climate Emergency Institute]
3,The PET Lamp project,NaN,The PET Lamp project is a beautiful melding of...,"[Goal 1, Goal 5, Goal 8, Goal 10, Goal 12, Goa...","[03 December, 2024, 03 December, 2024]","[Portfolio/Dossier of PET Lamp, A portfolio wh...",[Staff / Technical expertise],[Experienced and talented teams worldwide based],NaN,NaN,NaN,[AfricaEuropeAsia and PacificLatin America and...,"[Madrid, Spain]","[Chile, Colombia, Ethiopia, Ghana, Spain, Thai..."
4,Material Exchange Ventures AB,NaN,Material Exchange works with the world's leadi...,"[Goal 9, Goal 5, Goal 17, Goal 12, Goal 13]","[01 January, 1970]",[Digital Sourcing Report],"[Staff / Technical expertise, In-kind contribu...",[The Chief Sustainability Officer drives the t...,NaN,NaN,NaN,NaN,"[Stockholm, Sweden]",[\t\t\t\t\t\t\tN/A]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,North Sails,error,error,error,error,error,error,error,error,error,error,error,error,error
104,Vegan Fashion Week®,Vegan Fashion Week is a rebellion built on the...,Vegan Fashion Week is 501(c)(3) nonprofit orga...,"[Goal 4, Goal 8, Goal 11, Goal 12, Goal 13, Go...","[08 October, 2023]",[Vegan Fashion Week],"[Financing (in USD), Staff / Technical expertise]",[Goal: to source 1m in philanthropic funding t...,NaN,NaN,NaN,[Global],"[Los Angeles, California, USA]","[United States of America, France, United King..."
105,Haus von Eden,NaN,Haus von Eden is a pioneering digital platform...,"[Goal 5, Goal 12, Goal 13, Goal 17]","[31 December, 2025, 31 December, 2025, 31 Dece...",[Sign-up 50 companies for our sustainability m...,"[Financing (in USD), Staff / Technical expertise]","[110,000, 2 full-time Founder, 2 part-time emp...",NaN,NaN,NaN,[Europe],"[Hamburg, Germany]",[\t\t\t\t\t\t\tN/A]
106,Sellery,error,error,error,error,error,error,error,error,error,error,error,error,error


In [64]:
data.iloc[4,13]

['\t\t\t\t\t\t\tN/A']

In [65]:
i = 0

while i < len(data):
    j = 0
    while j < len(data.columns):
        if data.iloc[i,j] == ['\t\t\t\t\t\t\tN/A']:
            data.iloc[i,j] = np.NaN
        j += 1
    i += 1
data

,Case_Name,Description_Intro,Description_Description,SDGS&TARGETS,Delivery_Date,Delivery_Description,Resource_Name,Resource_Description,Progress_Title,Progress_Status,Progress_Filing,Region,Geographical Coverage,Countries
0,AlterContacts Academy for Sustainable Development,The AlterContacts Academy for Sustainable Deve...,The Academy tackles three key societal issues:...,"[Goal 4, Goal 8, Goal 9, Goal 17, Goal 10, Goa...","[31 December, 2024, 01 December, 2024, 20 Janu...",[Run one complete program in one of the partne...,"[Staff / Technical expertise, In-kind contribu...",[Full time coordinators from Think Tank AlterC...,[AlterContacts Academy for Sustainable Develop...,[On track],"[20 March, 2024]",[Global],[Multiple locations around the world],"[Albania, Colombia, Italy, Mexico, Netherlands..."
1,Embroidering Impact: A Harmony of Style and So...,NaN,The primary objective of the DooziStyle initia...,"[Goal 1, Goal 4, Goal 5, Goal 8, Goal 10, Goal...","[01 January, 2025, 01 January, 2030, 01 Januar...","[Artisan Network Expansion Program:, Diversifi...","[Financing (in USD), Staff / Technical experti...","[About $200,000 through a combination of self-...",NaN,NaN,NaN,[West Asia],"[Shirvan, Iran]",[Climate Emergency Institute]
2,"Needle to Narrative, Empowering Rural Communit...",NaN,"Dooziba, founded by Fatemeh Abdi Ghoushbolagh ...","[Goal 1, Goal 4, Goal 5, Goal 8, Goal 10, Goal...","[01 January, 2025, 01 January, 2027, 01 Januar...",[Expanding Artisan Network: Dooziba aims to ex...,"[Financing (in USD), Staff / Technical experti...","[Amount: $200,000 Description: Secured financi...",NaN,NaN,NaN,[West Asia],"[Shirvan, Iran]",[Climate Emergency Institute]
3,The PET Lamp project,NaN,The PET Lamp project is a beautiful melding of...,"[Goal 1, Goal 5, Goal 8, Goal 10, Goal 12, Goa...","[03 December, 2024, 03 December, 2024]","[Portfolio/Dossier of PET Lamp, A portfolio wh...",[Staff / Technical expertise],[Experienced and talented teams worldwide based],NaN,NaN,NaN,[AfricaEuropeAsia and PacificLatin America and...,"[Madrid, Spain]","[Chile, Colombia, Ethiopia, Ghana, Spain, Thai..."
4,Material Exchange Ventures AB,NaN,Material Exchange works with the world's leadi...,"[Goal 9, Goal 5, Goal 17, Goal 12, Goal 13]","[01 January, 1970]",[Digital Sourcing Report],"[Staff / Technical expertise, In-kind contribu...",[The Chief Sustainability Officer drives the t...,NaN,NaN,NaN,NaN,"[Stockholm, Sweden]",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,North Sails,error,error,error,error,error,error,error,error,error,error,error,error,error
104,Vegan Fashion Week®,Vegan Fashion Week is a rebellion built on the...,Vegan Fashion Week is 501(c)(3) nonprofit orga...,"[Goal 4, Goal 8, Goal 11, Goal 12, Goal 13, Go...","[08 October, 2023]",[Vegan Fashion Week],"[Financing (in USD), Staff / Technical expertise]",[Goal: to source 1m in philanthropic funding t...,NaN,NaN,NaN,[Global],"[Los Angeles, California, USA]","[United States of America, France, United King..."
105,Haus von Eden,NaN,Haus von Eden is a pioneering digital platform...,"[Goal 5, Goal 12, Goal 13, Goal 17]","[31 December, 2025, 31 December, 2025, 31 Dece...",[Sign-up 50 companies for our sustainability m...,"[Financing (in USD), Staff / Technical expertise]","[110,000, 2 full-time Founder, 2 part-time emp...",NaN,NaN,NaN,[Europe],"[Hamburg, Germany]",NaN
106,Sellery,error,error,error,error,error,error,error,error,error,error,error,error,error


In [66]:
data.to_excel('Example SDGs_data.xlsx')

In [57]:
driver.quit()